In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('gdrive/My Drive/CV/Face Detection/data/video/son.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# dnn module의 위치 정의
model_name = 'gdrive/My Drive/CV/Face Detection/data/res10_300x300_ssd_iter_140000.caffemodel'   # caffemodel의 weight 값
prototxt_name = 'gdrive/My Drive/CV/Face Detection/data/deploy.prototxt.txt'                     # model Architecture 에 대한 정보                
min_confidence = 0.5  # detection 으로 인정할 최소 확률(신뢰도)
file_name = 'gdrive/My Drive/CV/Face Detection/data/video/son.mp4' # Detection 할 원본 동영상
output_name = 'output_video.mp4'    # detection 된 output 동영상

In [ ]:
def detectAndDisplay(frame):
    # caffemodel의 weight 값과 모델 네트워크 구성을 불러와서 모델을 정의
    model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)

    # 이미지를 300x300 으로 size를 조정하고 blob 를 만든다.
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))

    model.setInput(blob)
    detections = model.forward()
    
    # detections 한 수만큼 루프가 돈다.
    for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]  # confidence 는 detection한 확률을 나타냄

            # min_confidence 보다 큰 경우에만 detection 으로 인정함
            if confidence > min_confidence:
                    (height, width) = frame.shape[:2]
                    # detection 된 영역을 boxing
                    # 상대적 좌표 * np.array([width, height, width, height]) 절대적인 boxing 좌표을 구해낸다. 
                    box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                    (startX, startY, endX, endY) = box.astype("int")
     
                    # 얼굴에 bounding box(사각형)를 그리고 확률값도 함께 나타낸다
                    text = "{:.2f}%".format(confidence * 100)
                    y = startY - 10 if startY - 10 > 10 else startY + 10
                    cv2.rectangle(frame, (startX, startY), (endX, endY),
                            (0, 255, 0), 2)
                    cv2.putText(frame, text, (startX, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (frame.shape[1], frame.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(frame)

In [ ]:
# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)

--(!) No captured frame -- Break!
